In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import gc
import re
import copy
import itertools
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm.notebook import tqdm
from datetime import datetime
import json,itertools
from typing import Optional
from glob import glob
import warnings
from IPython import display as ipd
warnings.filterwarnings("ignore")
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib as mpl
from matplotlib.patches import Rectangle
import seaborn as sns
import random
from joblib import Parallel, delayed
import os, shutil
import datetime 
import holidays
import dateutil.easter as easter

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet,SGDRegressor
from sklearn.model_selection import StratifiedGroupKFold

from xgboost import XGBRegressor
import xgboost as xgb
import lightgbm as lgb
from xgboost import plot_importance

from xgboost import plot_importance
from matplotlib import pyplot
import missingno as msno
import plotly.express as px

import optuna

# Keras
import tensorflow as tf

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline


# Keras
from tensorflow import keras
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model
from tensorflow.keras.layers import LSTM, Flatten, TimeDistributed, Conv1D, Input, Dense, Multiply, Add, Activation, GRU, BatchNormalization
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import load_model, save_model, Sequential
from tensorflow.data import Dataset
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Flatten,MaxPooling1D,BatchNormalization, Lambda, AveragePooling1D, Dropout, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
import tensorflow.keras as keras

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-sep-2022/train.csv', parse_dates=['date'])
original_train_df = train_df.copy()
test_df = pd.read_csv('../input/tabular-playground-series-sep-2022/test.csv', parse_dates=['date'])
train_df.head()

# EDA

In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

In [ ]:
train_df.describe()

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.head()

In [ ]:
day=pd.DatetimeIndex(train_df['date']).day
month=train_df['date'].dt.month_name()
year=pd.DatetimeIndex(train_df['date']).year
weekday=train_df['date'].dt.day_name()
quater= train_df['date'].dt.quarter

day1=pd.DatetimeIndex(test_df['date']).day
month1=test_df['date'].dt.month_name()
year1=pd.DatetimeIndex(test_df['date']).year
weekday1=test_df['date'].dt.day_name()
quater1= test_df['date'].dt.quarter

quater= train_df['date'].dt.quarter


In [ ]:
train_df.head()

In [ ]:
train_df.insert(loc=2, column='day', value=day)
train_df.insert(loc=3,column='weekday',value=weekday)
train_df.insert(loc=4, column='month', value=month)
train_df.insert(loc=5, column='quater', value=quater)
train_df.insert(loc=6, column='year', value=year)

In [ ]:
test_df.insert(loc=2, column='day', value=day1)
test_df.insert(loc=3,column='weekday',value=weekday1)
test_df.insert(loc=4, column='month', value=month1)
test_df.insert(loc=5, column='quater', value=quater1)
test_df.insert(loc=6, column='year', value=year1)


In [ ]:
train_df.head()

In [ ]:
plt.figure(figsize=(15,6))
train_df.groupby('year')['num_sold'].mean().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
train_df.groupby('store')['num_sold'].mean().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
sld_store = train_df.groupby(['store']).sum().reset_index()
plt.figure(figsize=(10,5))
ax = sns.barplot(x="store", y='num_sold', data=sld_store, palette="Blues")
ax.bar_label(ax.containers[0])
ax.set_title('Number sold in each store',
             fontsize = 14)

In [ ]:
plt.figure(figsize=(15,6))
train_df.groupby('country')['num_sold'].mean().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
sld_item= train_df.groupby(['product']).sum().reset_index()
plt.figure(figsize=(14,6))
ax = sns.barplot(x="num_sold", y='product', data=sld_item,color="darkblue")
ax.bar_label(ax.containers[0])
ax.set_title('Number sold of each product', fontsize = 18) 

In [ ]:
plt.figure(figsize=(15,6))
train_df.groupby('product')['num_sold'].mean().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
train_df.groupby(['year','store'])['num_sold'].mean().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
train_df.groupby(['year','product'])['num_sold'].mean().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
train_df.groupby(['year','country'])['num_sold'].mean().plot(kind='bar',stacked=True)
plt.show()

In [ ]:
plt.figure(figsize=(15,60))
train_df.groupby(['country','store','product'])['num_sold'].mean().unstack().plot(kind='bar',stacked=False)
plt.show()

In [ ]:
plt.figure(figsize=(20,500))
train_df.groupby(['country','product'])['num_sold'].mean().unstack().plot(kind='bar',stacked=False)
plt.show()

In [ ]:
f = plt.figure()
f.set_figwidth(20)
f.set_figheight(5)
train_df.groupby(['year','country','product'])['num_sold'].mean().unstack().plot(kind='bar',stacked=False)
plt.show()

In [ ]:
f = plt.figure()
f.set_figwidth(20)
f.set_figheight(5)
train_df.groupby(['country','product'])['num_sold'].mean().unstack().plot(kind='bar',stacked=False)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
train_gp = train_df.groupby('date').sum().reset_index()
plt.plot(train_gp['date'], train_gp['num_sold'])
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
for country in train_df['country'].unique():
    filt_train = train_df[train_df['country'] == country]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=country)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
for country in train_df['store'].unique():
    filt_train = train_df[train_df['store'] == country]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=country)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
for country in train_df['product'].unique():
    filt_train = train_df[train_df['product'] == country]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=country)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
for country in train_df['product'].unique():
    filt_train = train_df[train_df['product'] == country]

    train_gp = filt_train.groupby('year').sum().reset_index()
    plt.plot(train_gp['year'], train_gp['num_sold'], label=country)
plt.legend()
plt.show()

In [ ]:
weekly_df = train_df.groupby(["country","store", "product", pd.Grouper(key="date", freq="W")])["num_sold"].sum().rename("num_sold").reset_index()
monthly_df = train_df.groupby(["country","store", "product", pd.Grouper(key="date", freq="MS")])["num_sold"].sum().rename("num_sold").reset_index()
plt.figure(figsize=(15,6))
for product in train_df['product'].unique():
    filt_train = weekly_df[weekly_df['product'] == product]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=product)
plt.legend()
plt.show()

In [ ]:
monthly_df = train_df.groupby(["country","store", "product", pd.Grouper(key="date", freq="MS")])["num_sold"].sum().rename("num_sold").reset_index()
plt.figure(figsize=(15,6))
for product in train_df['product'].unique():
    filt_train = monthly_df[monthly_df['product'] == product]

    train_gp = filt_train.groupby('date').sum().reset_index()
    plt.plot(train_gp['date'], train_gp['num_sold'], label=product)
plt.legend()
plt.show()

In [ ]:
plt2= train_df.groupby(['country','month'])['num_sold'].sum().reset_index()
#plt2.reset_index(inplace=True)   
months = ["January", "February", "March", "April", "May", "June", 
          "July", "August", "September", "October", "November", "December"]
plt2['time'] = pd.Categorical(plt2['month'], categories=months, ordered=True)
plt2 = plt2.sort_values(['time','num_sold'], ascending= [True,False])
fig = px.line(plt2, x="time", y="num_sold", title= "Number sold products by month in each country",color="country", markers=True)
fig.show()

In [ ]:
plt2= train_df.groupby(['country','year'])['num_sold'].sum().reset_index()
#plt2.reset_index(inplace=True)   
years = [2017, 2018, 2019, 2020]
plt2['time'] = pd.Categorical(plt2['year'],categories=years, ordered=True)
plt2 = plt2.sort_values(['time','num_sold'], ascending= [True,False])
fig = px.line(plt2, x="time", y="num_sold", title= "Number sold products by month in each country",color="country", markers=True)
fig.show()

In [ ]:
plt4= train_df.groupby(['country','weekday'])['num_sold'].sum().reset_index().set_index('weekday')
plt4['Day_id'] = plt4.index
# plt4['Day_id'] = plt4['Day_id'].map(sorterIndex)
plt4.sort_values('Day_id', inplace=True)
fig = px.line(plt4, x=plt4.index, y="num_sold", title= "Total daily sold products in each country",color="country", markers=True)
fig.show()

In [ ]:
### plotdata preparation
sld_year = train_df.groupby(['year']).mean().reset_index()
sld_month = train_df.groupby(['month']).mean().reset_index().set_index('month')
sld_day = train_df.groupby(['weekday']).mean().reset_index().set_index('weekday')
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
sld_month= sld_month.reindex(new_order, axis=0)
sorter = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
sorterIndex = dict(zip(sorter,range(len(sorter))))
sld_day['Day_id'] = sld_day.index
sld_day['Day_id'] = sld_day['Day_id'].map(sorterIndex)
sld_day.sort_values('Day_id', inplace=True)

In [ ]:
sld_day.head()

In [ ]:
sld_month.head()

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(12, 16))
sns.lineplot(x=sld_year.year, y=sld_year.num_sold ,ax=ax[0], color='dodgerblue')
ax[0].set_title('year trend for number sold', fontsize=14)
ax[0].set_ylabel(ylabel='mean of number sold', fontsize=14)

sns.lineplot(x="month", y=sld_month.num_sold,data= sld_month , ax=ax[1], color='dodgerblue')
ax[1].set_title('month trend for number sold', fontsize=14)
ax[1].set_ylabel(ylabel='mean of number sold', fontsize=14)

sns.lineplot(x="weekday", y=sld_day.num_sold, data = sld_day , ax=ax[2], color='dodgerblue')
ax[2].set_title('day trend for number sold', fontsize=14)
ax[2].set_ylabel(ylabel='mean of number sold', fontsize=14)

plt.show()

In [ ]:
store_weights = train_df.groupby("store")["num_sold"].sum()/train_df["num_sold"].sum()
store_weights

In [ ]:
product_weights = train_df.groupby("product")["num_sold"].sum()/train_df["num_sold"].sum()
product_weights

In [ ]:
country_weights = train_df.groupby("country")["num_sold"].sum()/train_df["num_sold"].sum()
country_weights

In [ ]:
product_store_weights = train_df.groupby(["product","store"])["num_sold"].sum() / train_df.groupby(["product"])["num_sold"].sum()
product_store_weights

In [ ]:
product_country_weights = train_df.groupby(["product","country"])["num_sold"].sum() / train_df.groupby(["product"])["num_sold"].sum()
product_country_weights

In [ ]:
product_df = train_df.groupby(["date","product"])["num_sold"].sum().reset_index()
product_df.head()

In [ ]:
print("unique product",train_df['product'].unique())
print("Unique countrry",train_df['country'].unique())
print("unique store",train_df['store'].unique())

In [ ]:
train_df['num_sold'].plot()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
a= train_df.weekday.apply(lambda x : 1 if x in ['Saturday','Sunday'] else 0)
b= train_df['date'].dt.dayofweek
a,b
#train_df['dayofyear'] = train_df['date'].dt.dayofyear
#train_df['dayofmonth'] = train_df['date'].dt.days_in_month
c= np.arange(len(train_df.index)) # add a time step feature
#train['weekday'] = train['date'].dt.weekday

In [ ]:
train_df.head()

In [ ]:
a,b

In [ ]:
train_df["is_weekend"]= train_df.day.apply(lambda x : 1 if x in ['Saturday','Sunday'] else 0)
train_df['dayofweek'] = train_df['date'].dt.dayofweek
train_df['Time_step'] = np.arange(len(train_df.index))

test_df["is_weekend"]= test_df.day.apply(lambda x : 1 if x in ['Saturday','Sunday'] else 0)
test_df['dayofweek'] = test_df['date'].dt.dayofweek
test_df['Time_step'] = np.arange(len(test_df.index))

In [ ]:
train_df.head()

In [ ]:
train_data=train_df.drop(['row_id','weekday'],axis=1)
test_data=test_df.drop(['row_id','weekday'],axis=1)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_data.head()

In [ ]:
a = np.sin((train_data.dayofweek-1)*(2.*np.pi/12))
b = np.cos((train_data.dayofweek-1)*(2.*np.pi/12))

In [ ]:
le = LabelEncoder()
cols = [ "country","product","store","month" ,"year"]
for col in cols :
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.fit_transform(test_data[col])

In [ ]:
train_data.head()

In [ ]:
train_data1=train_data.copy()
test_data1=test_data.copy()

In [ ]:
train_data1.head()

In [ ]:
train_data['day_sin'] = np.sin(2 * np.pi * train_data['day'] / max(train_data['day']))
train_data['day_cos'] = np.cos(2 * np.pi * train_data['day'] / max(train_data['day']))
train_data['month_sin'] = np.sin(2 * np.pi * train_data['month'] / max(train_data['month']))
train_data['month_cos'] = np.cos(2 * np.pi * train_data['month'] / max(train_data['month']))
train_data['dayofweek_sin'] = np.sin(2 * np.pi * train_data['dayofweek'] / max(train_data['dayofweek']))
train_data['dayofweek_cos'] = np.cos(2 * np.pi * train_data['dayofweek'] / max(train_data['dayofweek']))

train_data['quater_sin'] = np.sin(2 * np.pi * train_data['quater'] / max(train_data['quater']))
train_data['quater_cos'] = np.cos(2 * np.pi * train_data['quater'] / max(train_data['quater']))

In [ ]:
test_data['day_sin'] = np.sin(2 * np.pi * test_data['day'] / max(test_data['day']))
test_data['day_cos'] = np.cos(2 * np.pi * test_data['day'] / max(test_data['day']))
test_data['month_sin'] = np.sin(2 * np.pi * test_data['month'] / max(test_data['month']))
test_data['month_cos'] = np.cos(2 * np.pi * test_data['month'] / max(test_data['month']))
test_data['dayofweek_sin'] = np.sin(2 * np.pi * test_data['dayofweek'] / max(test_data['dayofweek']))
test_data['dayofweek_cos'] = np.cos(2 * np.pi * test_data['dayofweek'] / max(test_data['dayofweek']))

test_data['quater_sin'] = np.sin(2 * np.pi * test_data['quater'] / max(test_data['quater']))
test_data['quater_cos'] = np.cos(2 * np.pi * test_data['quater'] / max(test_data['quater']))

In [ ]:
train_data.head()

In [ ]:
train_data1.head()

In [ ]:
train_data.drop(['day','month','dayofweek'],axis=1,inplace=True)
test_data.drop(['day','month','dayofweek'],axis=1,inplace=True)

In [ ]:
train_data.head()

In [ ]:
X_train = train_data[train_data.date.between('2017-01-01', '2020-12-31')].copy()
X_val = train_data[train_data.date.between('2017-11-01', '2020-12-31')].copy()
# X_train1 = train_data1[train_data1.date.between('2017-01-01', '2020-10-31')].copy()
# X_val1 = train_data1[train_data1.date.between('2020-11-01', '2020-12-31')].copy()

In [ ]:
X_train.drop(['date'],axis=1,inplace=True)
X_val.drop(['date'],axis=1,inplace=True)
# X_train1.drop(['date'],axis=1,inplace=True)
# X_val1.drop(['date'],axis=1,inplace=True)

In [ ]:
X_train

In [ ]:
y_train=X_train["num_sold"]
X_train = X_train.drop("num_sold",axis=1)
y_val= X_val["num_sold"]
X_val= X_val.drop("num_sold",axis=1)


In [ ]:
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(n_estimators=1000)

In [ ]:
from xgboost import XGBRegressor

class CustomXGBRegressor(XGBRegressor):
    
    def __init__(self, **params):
        
        super().__init__(**params)
        self.eval_set = params['eval_set']
    
    def fit(self, X, y):
        super().fit(X, y, eval_set=self.eval_set, verbose=100) # output progress every 100 iterations

In [ ]:
xgb = CustomXGBRegressor(n_estimators = 1000, early_stopping_rounds = 20, learning_rate = 0.035, eval_set = [(X_val, y_val)])

In [ ]:
cols = [col for col in X_train.columns]
cols

In [ ]:
X_train

In [ ]:
from sklearn.ensemble import VotingRegressor

model = VotingRegressor(estimators=[
    ('xgb', xgb), ('rf', rf), ('gb', gb)])

model.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
val_pred= model.predict(X_val)

errors = mean_squared_error(val_pred,y_val)
# report error
print(errors)

In [ ]:
cols = [col for col in X.columns]
cols

In [ ]:
test_data.drop('date',axis=1,inplace=True)
cols = [col for col in test_data.columns]

In [ ]:
X_train.head()

In [ ]:
test_data.head()

In [ ]:
preds = model.predict(test_data)

In [ ]:
preds[:5]

In [ ]:
submission = pd.DataFrame({
    'row_id' : test_df['row_id'],
    'num_sold' : preds
})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index= False)

In [ ]:
from IPython.display import FileLink
FileLink('submission.csv')